In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Tải bộ dữ liệu từ HuggingFace Datasets
from datasets import load_dataset

# Load dataset NTC-SCV
ds = load_dataset('thainq107/ntc-scv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

# Tên mô hình BERT
model_name = "distilbert-base-uncased"  # Hoặc "bert-base-uncased"

# Khởi tạo tokenizer từ mô hình pretrained
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True  # Sử dụng tokenizer nhanh hơn
)

# Độ dài tối đa của chuỗi
max_seq_length = 100
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

# Hàm xử lý dữ liệu
def preprocess_function(examples):
    # Tokenize các văn bản
    result = tokenizer(
        examples["preprocessed_sentence"],  # Cột chứa văn bản đã tiền xử lý
        padding="max_length",  # Đệm chuỗi đến max_seq_length
        max_length=max_seq_length,
        truncation=True  # Cắt bớt chuỗi nếu dài hơn max_seq_length
    )
    result["label"] = examples['label']  # Gán nhãn tương ứng
    return result

# Tiền xử lý tập dữ liệu bằng tokenizer
processed_dataset = ds.map(
    preprocess_function,
    batched=True,  # Xử lý theo batch để tăng tốc độ
    desc="Running tokenizer on dataset",
)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Running tokenizer on dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Số nhãn trong bài toán phân loại
num_labels = 2

# Cấu hình mô hình
config = AutoConfig.from_pretrained(
    model_name,  # Tên mô hình đã được định nghĩa trước đó (ví dụ: "distilbert-base-uncased")
    num_labels=num_labels,  # Số nhãn trong bài toán phân loại
    finetuning_task="text-classification"  # Nhiệm vụ fine-tuning
)

# Khởi tạo mô hình BERT cho bài toán phân loại
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config  # Sử dụng cấu hình đã định nghĩa
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [6]:
import numpy as np
import evaluate

# Tải metric "accuracy" từ thư viện evaluate
metric = evaluate.load("accuracy")

# Hàm tính toán các chỉ số đánh giá
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Lấy nhãn dự đoán có xác suất cao nhất
    predictions = np.argmax(predictions, axis=1)
    # Tính toán độ chính xác
    result = metric.compute(predictions=predictions, references=labels)
    return result


In [7]:
# Chia tập train thành train và validation (80:20)
split_dataset = processed_dataset["train"].train_test_split(test_size=0.2)

# Thêm vào DatasetDict
processed_dataset = {
    "train": split_dataset["train"],
    "validation": split_dataset["test"],
    "test": processed_dataset["test"]
}

# Chuyển đổi thành DatasetDict
from datasets import DatasetDict
processed_dataset = DatasetDict(processed_dataset)

# Kiểm tra kết quả
print(processed_dataset)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})


In [8]:
from transformers import TrainingArguments, Trainer

# Cấu hình tham số huấn luyện
training_args = TrainingArguments(
    output_dir="save_model",  # Thư mục lưu mô hình
    learning_rate=2e-5,  # Tốc độ học
    per_device_train_batch_size=128,  # Kích thước batch cho tập train
    per_device_eval_batch_size=128,  # Kích thước batch cho tập eval
    num_train_epochs=10,  # Số epoch
    eval_strategy="epoch",  # Đánh giá trên tập validation mỗi epoch
    save_strategy="epoch",  # Lưu mô hình tốt nhất mỗi epoch
    load_best_model_at_end=True  # Tải mô hình tốt nhất sau khi huấn luyện
)

# Tạo Trainer
trainer = Trainer(
    model=model,  # Mô hình đã được khởi tạo
    args=training_args,  # Tham số huấn luyện
    train_dataset=processed_dataset["train"],  # Dữ liệu huấn luyện
    eval_dataset=processed_dataset["validation"],  # Dữ liệu validation
    compute_metrics=compute_metrics,  # Hàm tính toán chỉ số đánh giá
    tokenizer=tokenizer,  # Tokenizer
)

# Huấn luyện mô hình
trainer.train()


<ipython-input-8-798837cb8f1d>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.449597,0.795333
2,No log,0.426167,0.807000
3,0.452500,0.386564,0.830833
4,0.452500,0.384594,0.831000
5,0.452500,0.379804,0.836667
6,0.328700,0.400635,0.833000
7,0.328700,0.415253,0.836000
8,0.259600,0.420396,0.836000
9,0.259600,0.428766,0.835167
10,0.259600,0.428752,0.835167


TrainOutput(global_step=1880, training_loss=0.32096468742857587, metrics={'train_runtime': 2125.1632, 'train_samples_per_second': 112.933, 'train_steps_per_second': 0.885, 'total_flos': 6209409312000000.0, 'train_loss': 0.32096468742857587, 'epoch': 10.0})